In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

# keras已内置了cifar10数据集
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 206s 1us/step


In [2]:
# 探索CIFAR-10数据集
print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
# 查看第一张图片及其标签
print('X_train[0]:', X_train[0], '\ny_train[0]:', y_train[0])

X_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000, 1)
X_train[0]: [[[ 59  62  63]
  [ 43  46  45]
  [ 50  48  43]
  ..., 
  [158 132 108]
  [152 125 102]
  [148 124 103]]

 [[ 16  20  20]
  [  0   0   0]
  [ 18   8   0]
  ..., 
  [123  88  55]
  [119  83  50]
  [122  87  57]]

 [[ 25  24  21]
  [ 16   7   0]
  [ 49  27   8]
  ..., 
  [118  84  50]
  [120  84  50]
  [109  73  42]]

 ..., 
 [[208 170  96]
  [201 153  34]
  [198 161  26]
  ..., 
  [160 133  70]
  [ 56  31   7]
  [ 53  34  20]]

 [[180 139  96]
  [173 123  42]
  [186 144  30]
  ..., 
  [184 148  94]
  [ 97  62  34]
  [ 83  53  34]]

 [[177 144 116]
  [168 129  94]
  [179 142  87]
  ..., 
  [216 184 140]
  [151 118  84]
  [123  92  72]]] 
y_train[0]: [6]


In [3]:
# 数据预处理

# 常量定义：高度、宽度、分类的数目
height, width, nb_class = 32, 32, 10

# 对图片X部分做归一化
X_train = X_train / 255.
X_test = X_test / 255.

# 处理标签
y_train = np_utils.to_categorical(y_train, nb_class)
y_test = np_utils.to_categorical(y_test, nb_class)

# 图像数据生成器
gen = ImageDataGenerator()

# 打印数据信息，比较与之前的差别
print('X_train.shape:', X_train.shape)
print('y_train.shape:', y_train.shape)
print('X_train[0]:', X_train[0], '\ny_train[0]:', y_train[0])

X_train.shape: (50000, 32, 32, 3)
y_train.shape: (50000, 10)
X_train[0]: [[[ 0.23137255  0.24313725  0.24705882]
  [ 0.16862745  0.18039216  0.17647059]
  [ 0.19607843  0.18823529  0.16862745]
  ..., 
  [ 0.61960784  0.51764706  0.42352941]
  [ 0.59607843  0.49019608  0.4       ]
  [ 0.58039216  0.48627451  0.40392157]]

 [[ 0.0627451   0.07843137  0.07843137]
  [ 0.          0.          0.        ]
  [ 0.07058824  0.03137255  0.        ]
  ..., 
  [ 0.48235294  0.34509804  0.21568627]
  [ 0.46666667  0.3254902   0.19607843]
  [ 0.47843137  0.34117647  0.22352941]]

 [[ 0.09803922  0.09411765  0.08235294]
  [ 0.0627451   0.02745098  0.        ]
  [ 0.19215686  0.10588235  0.03137255]
  ..., 
  [ 0.4627451   0.32941176  0.19607843]
  [ 0.47058824  0.32941176  0.19607843]
  [ 0.42745098  0.28627451  0.16470588]]

 ..., 
 [[ 0.81568627  0.66666667  0.37647059]
  [ 0.78823529  0.6         0.13333333]
  [ 0.77647059  0.63137255  0.10196078]
  ..., 
  [ 0.62745098  0.52156863  0.2745098 ]
  

In [6]:
# 搭建神经网络

# 生成Sequantial 模型
model = Sequential()

# 添加输入层
model.add(Conv2D(32, 3, padding='same', input_shape=X_train.shape[1:], activation='relu'))
# 添加卷积层
model.add(Conv2D(32, 3, activation='relu'))
# 添加池化层
model.add(MaxPool2D(2))
# 添加失活层Dropout，有效防止过拟合
model.add(Dropout(0.25))

# 之后依然是卷积、池化与失活
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.25))
# 添加一个 Flatten 层，或者叫展开层，它可将高维的张量展开，变成一个一维张量（即向量）
model.add(Flatten())

# 之后是全连接层、失活层，以及输出层
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(nb_class, activation='softmax'))
model.add(Dense(nb_class, activation='sigmoid'))

# 查看搭建的神经网络的信息
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [7]:
# 使用凸优化模块训练模型

# 使用 categorical_crossentropy 作为损失函数，或者叫目标函数，这是一个多分类中比较常用的损失函数
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# 调用模型的训练方法
batch_size = 32
model.fit_generator(gen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=20, verbose=1, validation_data=(X_test, y_test))

Epoch 1/20
1562/1562 [==============================] - 454s 291ms/step - loss: 1.6234 - acc: 0.4052 - val_loss: 1.2782 - val_acc: 0.5316
Epoch 2/20
1562/1562 [==============================] - 461s 295ms/step - loss: 1.2259 - acc: 0.5614 - val_loss: 1.0409 - val_acc: 0.6305
Epoch 3/20
1562/1562 [==============================] - 462s 296ms/step - loss: 1.0718 - acc: 0.6198 - val_loss: 0.9718 - val_acc: 0.6573
Epoch 4/20
1562/1562 [==============================] - 451s 289ms/step - loss: 0.9736 - acc: 0.6555 - val_loss: 0.8782 - val_acc: 0.6899
Epoch 5/20
1562/1562 [==============================] - 454s 291ms/step - loss: 0.9009 - acc: 0.6831 - val_loss: 0.8204 - val_acc: 0.7148
Epoch 6/20
1562/1562 [==============================] - 474s 304ms/step - loss: 0.8391 - acc: 0.7041 - val_loss: 0.7632 - val_acc: 0.7344
Epoch 7/20
1562/1562 [==============================] - 463s 297ms/step - loss: 0.7969 - acc: 0.7192 - val_loss: 0.7377 - val_acc: 0.7482
Epoch 8/20
1562/1562 [============

In [8]:
# 使用模型进行预测

# 选取测试集 X_test 的前 10 张图片，用训练好的模型进行预测
y_preds = model.predict(X_test[:10])

# 创建一个字典，将标签的值和分类的名字对应起来，方便查看
cifar10_cats = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog',
                6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

# 对比真实标签和预测结果
for i in range(10):
    print('正确结果: {}\n预测结果: {}\n'.format(cifar10_cats[np.argmax(y_test[i])], 
                                            cifar10_cats[np.argmax(y_preds[i])]))

正确结果: cat
预测结果: cat

正确结果: ship
预测结果: ship

正确结果: ship
预测结果: ship

正确结果: airplane
预测结果: airplane

正确结果: frog
预测结果: frog

正确结果: frog
预测结果: frog

正确结果: automobile
预测结果: automobile

正确结果: frog
预测结果: frog

正确结果: cat
预测结果: cat

正确结果: automobile
预测结果: automobile

